In [ ]:
!git clone https://github.com/Yaswanth-B/AccessibleLLM.git

In [ ]:
!pip install -q datasets

In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("arian2502/firstdataset", split="train")

In [ ]:
!pip install bert-score torch transformers

In [ ]:
!pip install rouge

In [ ]:
!pip install nltk
import nltk
nltk.download('punkt')

In [7]:
from rouge import Rouge

from datasets import load_dataset

dataset = load_dataset("arian2502/firstdataset", split="train")
true_text = []

for data_item in dataset:
    # Assuming your dataset has a key "caption" for captions
    caption = data_item["text"]
    true_text.append(caption)


with open("/content/AccessibleLLM/object_detection/generated_captions.txt", "r") as model1_file:
    model1_captions = model1_file.readlines()

with open("/content/AccessibleLLM/object_detection/generated_captions(trained).txt", "r") as model2_file:
    model2_captions = model2_file.readlines()

# Initialize ROUGE
rouge = Rouge()

# Compute ROUGE scores for blip untrained
scores_model1 = rouge.get_scores(model1_captions, true_text, avg=True)
print("ROUGE Scores for BLIP(untrained):")
print(scores_model1)

# Compute ROUGE scores for blip trained
scores_model2 = rouge.get_scores(model2_captions, true_text, avg=True)
print("ROUGE Scores for BLIP(trained)t:")
print(scores_model2)

# Calculate precision, recall, and F1-score based on ROUGE scores
precision_model1 = scores_model1["rouge-l"]["p"]
recall_model1 = scores_model1["rouge-l"]["r"]
f1_model1 = scores_model1["rouge-l"]["f"]

precision_model2 = scores_model2["rouge-l"]["p"]
recall_model2 = scores_model2["rouge-l"]["r"]
f1_model2 = scores_model2["rouge-l"]["f"]

print("Precision, Recall, and F1-score for BLIP(untrained):")
print("Precision:", precision_model1)
print("Recall:", recall_model1)
print("F1-score:", f1_model1)

print("Precision, Recall, and F1-score for BLIP(trained):")
print("Precision:", precision_model2)
print("Recall:", recall_model2)
print("F1-score:", f1_model2)

ROUGE Scores for BLIP(untrained):
{'rouge-1': {'r': 0.38623084720771134, 'p': 0.26676010739518513, 'f': 0.3066864136993266}, 'rouge-2': {'r': 0.1298950742068691, 'p': 0.0725541792011556, 'f': 0.08984282440623012}, 'rouge-l': {'r': 0.3364418769543522, 'p': 0.23059880787255438, 'f': 0.2657411209552809}}
ROUGE Scores for BLIP(trained)t:
{'rouge-1': {'r': 0.7300539801249587, 'p': 0.6806757571022624, 'f': 0.6995404673734046}, 'rouge-2': {'r': 0.6493630657374209, 'p': 0.577443416297378, 'f': 0.6069535829964249}, 'rouge-l': {'r': 0.7226284310678925, 'p': 0.6730824339010327, 'f': 0.6922038904392883}}
Precision, Recall, and F1-score for BLIP(untrained):
Precision: 0.23059880787255438
Recall: 0.3364418769543522
F1-score: 0.2657411209552809
Precision, Recall, and F1-score for BLIP(trained):
Precision: 0.6730824339010327
Recall: 0.7226284310678925
F1-score: 0.6922038904392883


In [9]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import bert_score

from datasets import load_dataset

dataset = load_dataset("arian2502/firstdataset", split="train")
true_text = []

with open("/content/AccessibleLLM/object_detection/generated_captions.txt", "r") as file:
    model1_text = file.read()

blip_untrained = model1_text.strip().split('\n')

with open("/content/AccessibleLLM/object_detection/generated_captions(trained).txt", "r") as file:
    model2_text = file.read()

blip_trained = model2_text.strip().split('\n')

for data_item in dataset:
    # Assuming your dataset has a key "caption" for captions
    caption = data_item["text"]
    true_text.append(caption)

# Compute BERTScore for blip(untrained) Model
P_A, R_A, F1_A = bert_score.score(
    blip_untrained,
    true_text,
    lang='en',  # Language of the captions (e.g., 'en' for English)
    model_type='bert-base-uncased',  # BERT model type
)

# Compute BERTScore for blip(trained) Model
P_B, R_B, F1_B = bert_score.score(
    blip_trained,
    true_text,
    lang='en',  # Language of the captions (e.g., 'en' for English)
    model_type='bert-base-uncased',  # BERT model type
)

print("BERTScore for BLIP(untrained) Model:")
print(f"Precision: {P_A.mean():.4f}")
print(f"Recall: {R_A.mean():.4f}")
print(f"F1-score: {F1_A.mean():.4f}")

print("\nBERTScore for BLIP(trained) Model:")
print(f"Precision: {P_B.mean():.4f}")
print(f"Recall: {R_B.mean():.4f}")
print(f"F1-score: {F1_B.mean():.4f}")


BERTScore for BLIP(untrained) Model:
Precision: 0.5309
Recall: 0.6051
F1-score: 0.5637

BERTScore for BLIP(trained) Model:
Precision: 0.8848
Recall: 0.8886
F1-score: 0.8854


In [10]:
import nltk
from datasets import load_dataset

dataset = load_dataset("arian2502/firstdataset", split="train")
true_text = []

def compute_gleu_score(generated_caption, reference_caption):
    gen_tokens = nltk.word_tokenize(generated_caption.lower())
    ref_tokens = nltk.word_tokenize(reference_caption.lower())
    gleu_score = nltk.translate.gleu_score.sentence_gleu([ref_tokens], gen_tokens)
    return gleu_score

def compute_average_gleu_scores(model_captions, reference_captions):
    total_gleu_score = 0.0
    num_captions = len(model_captions)

    for i in range(num_captions):
        model_caption = model_captions[i]
        ref_caption = reference_captions[i]
        gleu_score = compute_gleu_score(model_caption, ref_caption)
        total_gleu_score += gleu_score

    average_gleu_score = total_gleu_score / num_captions
    return average_gleu_score

def read_captions_from_file(file_name):
    with open(file_name, 'r') as file:
        captions = file.read().split('\n')
    return [caption.strip() for caption in captions if caption.strip()]

for data_item in dataset:
    # Assuming your dataset has a key "caption" for captions
    caption = data_item["text"]
    true_text.append(caption)

captions_model_a = read_captions_from_file("/content/AccessibleLLM/object_detection/generated_captions.txt")
captions_model_b = read_captions_from_file("/content/AccessibleLLM/object_detection/generated_captions(trained).txt")

# Compute average GLEU scores for Model A and Model B
avg_gleu_model_a = compute_average_gleu_scores(captions_model_a, true_text)
avg_gleu_model_b = compute_average_gleu_scores(captions_model_b, true_text)

# Print the results
print("Average GLEU Score for BLIP(untrained):", avg_gleu_model_a)
print("Average GLEU Score for BLIP(trained):", avg_gleu_model_b)

# Determine which model performs better
if avg_gleu_model_a > avg_gleu_model_b:
    print("BLIP(untrained) performs better.")
elif avg_gleu_model_b > avg_gleu_model_a:
    print("BLIP(trained) performs better.")
else:
    print("Both models perform equally well.")


Average GLEU Score for BLIP(untrained): 0.08593777080376051
Average GLEU Score for BLIP(trained): 0.6954368111617628
BLIP(trained) performs better.


In [11]:
import nltk
from nltk.translate.bleu_score import corpus_bleu

from datasets import load_dataset

dataset = load_dataset("arian2502/firstdataset", split="train")
true_text = []

def read_captions_from_file(file_name):
    with open(file_name, 'r') as file:
        captions = file.read().split('\n')
    return [caption.strip() for caption in captions if caption.strip()]

for data_item in dataset:
    # Assuming your dataset has a key "caption" for captions
    caption = data_item["text"]
    true_text.append(caption)

captions_model_a = read_captions_from_file("/content/AccessibleLLM/object_detection/generated_captions.txt")
captions_model_b = read_captions_from_file("/content/AccessibleLLM/object_detection/generated_captions(trained).txt")

# Tokenize captions
model1_captions = [nltk.word_tokenize(caption.lower()) for caption in captions_model_a]
model2_captions = [nltk.word_tokenize(caption.lower()) for caption in captions_model_b]
reference_captions = [[nltk.word_tokenize(caption.lower())] for caption in true_text]

# Calculate BLEU score for each model
bleu_score_model1 = corpus_bleu(reference_captions, model1_captions)
bleu_score_model2 = corpus_bleu(reference_captions, model2_captions)

print("BLEU Score for BLIP(untrained):", bleu_score_model1)
print("BLEU Score for BLIP(trained):", bleu_score_model2)


BLEU Score for BLIP(untrained): 0.04215093904464002
BLEU Score for BLIP(trained): 0.704256378969982
